In [1]:
import os
os.chdir('..')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


> # Forecasting future sales for Rossmann Pharmaceuticals across various stores

>>## EDA Analysis

>>> ### Load Dataset 

In [2]:
train_data_path = 'Data/train.csv'
test_data_path ='Data/test.csv'
store_data_path ='Data/store.csv'

In [3]:

from scripts.data_loader import load_data

In [4]:
train_data, test_data, store_data = load_data(train_data_path,test_data_path,store_data_path)


2025-01-02 09:52:31,498 - INFO - Loading data from file...
2025-01-02 09:52:32,364 - INFO - Train Data, Test Data and Store Data loaded with shape (1017209, 9), (41088, 8), and (1115, 10) respectively
